In [1]:
#importing the libraries
import pandas as pd
import numpy as np
%pip install matplotlib
import matplotlib.pyplot as plt
%pip install seaborn
import seaborn as sns
%pip install scikit-learn
%pip install tensorflow
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
%pip install lightgbm
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
#  Connect to PostgreSQL & Load Data
DB_USER = "myuser"
DB_PASSWORD = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydatabase"

connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)

query = 'SELECT * FROM preprocessed_data'
df = pd.read_sql_query(query, con=engine)

print("\n✅ Data loaded successfully!")
print("Dataset Shape:", df.shape)
print("Columns:", df.columns)


✅ Data loaded successfully!
Dataset Shape: (32732, 28)
Columns: Index(['engine_id', 'cycle', 'op_setting_1', 'op_setting_2', 'op_setting_3',
       'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6',
       'sensor_7', 'sensor_8', 'sensor_9', 'sensor_10', 'sensor_11',
       'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15', 'sensor_16',
       'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20', 'sensor_21',
       'dataset_name', 'source'],
      dtype='object')


In [4]:
# Compute Remaining Useful Life (RUL)
df['RUL'] = df.groupby('engine_id')['cycle'].transform(max) - df['cycle']

# Optional: Define failure probability (assuming threshold-based classification)
df['failure_probability'] = df['RUL'].apply(lambda x: 1 if x <= 20 else 0)  # Example threshold of 20 cycles

print("\n✅ RUL and Failure Probability computed successfully!")
print(df[['engine_id', 'cycle', 'RUL', 'failure_probability']].head())



✅ RUL and Failure Probability computed successfully!
   engine_id  cycle  RUL  failure_probability
0          1      1  191                    0
1          1      2  190                    0
2          1      3  189                    0
3          1      4  188                    0
4          1      5  187                    0


C:\Users\pc\AppData\Local\Temp\ipykernel_8828\1295633808.py:2: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df['RUL'] = df.groupby('engine_id')['cycle'].transform(max) - df['cycle']


In [5]:
# Feature Engineering
X = df.drop(columns=['RUL', 'failure_probability','dataset_name',"source"], errors='ignore')
y_rul = df['RUL']  # Target for RUL prediction
y_fail = df['failure_probability'] 

# Train-test split (80-20)
X_train, X_test, y_rul_train, y_rul_test = train_test_split(X, y_rul, test_size=0.2, random_state=42)

print("\n✅ Data split into train and test sets.")
print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)


✅ Data split into train and test sets.
Training set shape: (26185, 26)
Test set shape: (6547, 26)


In [11]:
#  SVR
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Random Forest 
rf_rul = RandomForestRegressor(n_estimators=100, random_state=42)
rf_rul.fit(X_train, y_rul_train)

# Train XGBoost 
xgb_rul = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
xgb_rul.fit(X_train, y_rul_train)

# Train SVR
svr_rul = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
svr_rul.fit(X_train_scaled, y_rul_train)

# Train LightGBM 
lgb_rul = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.01, random_state=42)
lgb_rul.fit(X_train, y_rul_train)

print("\n✅ Models trained successfully!")

calculations = {
    "Random Forest (RUL)": {
        "MAE": 0.0,
        "RMSE": 0.0,
        "R² Score": 0.0,
        "Accuracy": 0.0
    },
    "XGBoost (RUL)": {
        "MAE": 0.0,
        "RMSE": 0.0,
        "R² Score": 0.0,
        "Accuracy": 0.0
    },
    "SVR (RUL)": {
        "MAE": 0.0,
        "RMSE": 0.0,
        "R² Score": 0.0,
        "Accuracy": 0.0
    },
    "LightGBM (RUL)": {
        "MAE": 0.0,
        "RMSE": 0.0,
        "R² Score": 0.0,
        "Accuracy": 0.0
    },
}


def regression_accuracy(y_true, y_pred, tolerance=0.1):
    
    within_tolerance = abs(y_true - y_pred) / y_true <= tolerance
    return within_tolerance.mean()

def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    print(f"\n🔹 {model_name} Performance:")
    print(f"  - MAE: {mean_absolute_error(y_test, y_pred):.4f}")
    calculations[model_name]["MAE"] = mean_absolute_error(y_test, y_pred)


    rmse = np.sqrt(mean_squared_error(y_test, y_pred)) 
    print(f"  - RMSE: {rmse:.4f}")
    print(f"  - R² Score: {r2_score(y_test, y_pred):.4f}")
    calculations[model_name]["RMSE"] = rmse
    calculations[model_name]["R² Score"] = r2_score(y_test, y_pred)

    accuracy = regression_accuracy(y_test, y_pred)
    print(f"  - Accuracy (within 10% tolerance): {accuracy:.2%}")
    calculations[model_name]["Accuracy"] = accuracy
    return y_pred

# Evaluate models on test data
y_rul_pred_rf = evaluate_model(rf_rul, X_test, y_rul_test, "Random Forest (RUL)")
y_rul_pred_xgb = evaluate_model(xgb_rul, X_test, y_rul_test, "XGBoost (RUL)")
y_rul_pred_svr = evaluate_model(svr_rul, X_test_scaled, y_rul_test, "SVR (RUL)")
y_rul_pred_lgb = evaluate_model(lgb_rul, X_test, y_rul_test, "LightGBM (RUL)")                      

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001042 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3184
[LightGBM] [Info] Number of data points in the train set: 26185, number of used features: 18
[LightGBM] [Info] Start training from score 119.091083

✅ Models trained successfully!

🔹 Random Forest (RUL) Performance:
  - MAE: 7.7917
  - RMSE: 12.4993
  - R² Score: 0.9672
  - Accuracy (within 10% tolerance): 67.18%

🔹 XGBoost (RUL) Performance:
  - MAE: 5.5723
  - RMSE: 7.6938
  - R² Score: 0.9876
  - Accuracy (within 10% tolerance): 77.46%

🔹 SVR (RUL) Performance:
  - MAE: 27.8765
  - RMSE: 39.7768
  - R² Score: 0.6679
  - Accuracy (within 10% tolerance): 28.00%

🔹 LightGBM (RUL) Performance:
  - MAE: 31.9470
  - RMSE: 40.2227
  - R² Score: 0.6604
  - Accuracy (within 10% tolerance): 24.82%


saving results

In [13]:
import json
from sqlalchemy import create_engine

# Step 5: Define database connection
DB_USER = "myuser"
DB_PASSWORD = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydatabase"

# Construct the connection string
connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create the SQLAlchemy engine
engine = create_engine(connection_string)


# Step 6: Save preprocessed data to PostgreSQL
table_name = "trainning_data_results"

model_results = [
    {
        "model": "Random Forest (RUL)",
        "calculations": json.dumps({k: float(v) if isinstance(v, np.float64) else v for k, v in calculations["Random Forest (RUL)"].items()}),  # Convert np.float64 to standard float
        "predictions": json.dumps(y_rul_pred_rf.tolist())  # Convert list to JSON
    },
    {
        "model": "XGBoost (RUL)",
        "calculations": json.dumps({k: float(v) if isinstance(v, np.float64) else v for k, v in calculations["XGBoost (RUL)"].items()}),
        "predictions": json.dumps(y_rul_pred_xgb.tolist())
    },
    {
        "model": "SVR (RUL)",
        "calculations": json.dumps({k: float(v) if isinstance(v, np.float64) else v for k, v in calculations["SVR (RUL)"].items()}),
        "predictions": json.dumps(y_rul_pred_svr.tolist())
    },
    {
        "model": "LightGBM (RUL)",
        "calculations": json.dumps({k: float(v) if isinstance(v, np.float64) else v for k, v in calculations["LightGBM (RUL)"].items()}),
        "predictions": json.dumps(y_rul_pred_lgb.tolist())
    }
]
df_model_results = pd.DataFrame(model_results)

# Save the preprocessed data to the database
df_model_results.to_sql(table_name, engine, if_exists="replace", index=False)

print(f"✅ Model results successfully stored in PostgreSQL table '{table_name}'.")

✅ Model results successfully stored in PostgreSQL table 'trainning_data_results'.
